In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier
from itertools import chain, combinations

In [2]:
import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [3]:
dataset='qanda'
datasets = [dataset]

In [11]:
fr_groundtruths =['URLa_FR', 'URLb_FR']
lr_groundtruths =['HASHTAG','URL_LR','POLITICIAN_1H_LR', 'PARTY_FOLLOWER_LR']

In [5]:
performance = {dataset : defaultdict(dict) for dataset in datasets}
gt_f1 = {dataset : defaultdict(dict) for dataset in datasets}
overlap_f1 = {dataset : defaultdict(dict) for dataset in datasets}

In [6]:
import itertools
def all_combinations(l):
    return(itertools.product(l, repeat=2))

In [7]:
def load_vector(fp):
    with open(fp, 'rb') as rf:
        vec = pk.load(rf)
    return vec

In [8]:
# granularity = '_per_user'
# feature = 'use'

# for dataset in datasets:
#     for train, test in itertools.chain(all_combinations(lr_groundtruths),all_combinations(fr_groundtruths)):
#         print(dataset, train, test)
#         train_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+train+'_'+granularity+'.pk')
#         if not os.path.exists(train_path):
#             continue
#         test_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+test+'_'+granularity+'.pk')
#         if not os.path.exists(test_path):
#             continue

#         y_train = load_vector(train_path).values
#         mask_train = y_train >= 0

#         y_test = load_vector(test_path).values
#         mask_test = y_test >= 0
#         mask = mask_train | mask_test
        
#         # overlap_mask = mask_train & mask_test
#         # print("Overlap F1:", f1_score(y_train[overlap_mask], y_test[overlap_mask],average='macro'))
#         # overlap_f1[dataset][train][test] = f1_score(y_train[overlap_mask], y_test[overlap_mask],average='macro')
#         # if train == 'POLITICIAN_LR' and test == 'POLITICIAN_1H_LR':
#         #     mask = mask_train & mask_test
#         if any(mask):
#             # gt_f1[dataset][train][test] = f1_score(y_train[mask], y_test[mask],average='macro')

#             y = y_train[mask]
#             # y = y_train[mask_train]
            
#             feature_path = os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk')
#             X_orig = load_vector(feature_path)
            
#             X = X_orig[mask]
#             # X = X_orig[mask_train]
#             et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
#             if train == test:
#                 continue
#                 # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
#                 # validation instance
#                 skf = StratifiedKFold(n_splits=5, shuffle=True)
#                 # getting the model validation predictions
#                 preds = cross_val_predict(et, X, y, cv=skf, method='predict_proba')
#             else:
#                 # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
#                 et.fit(X,y)
#                 preds = et.predict_proba(X)
#                 print
#             if train in fr_groundtruths:
#                 preds = preds[:,1]
#             # evaluating the model
#             print(dataset, train, test)
#             print()
#             try:
#                 if test == 'POLITICIAN_LR' and train == 'POLITICIAN_1H_LR':
#                     N = preds.shape[0]
#                     preds = np.c_[np.zeros(N),preds]
#                 if (dataset, train, test) == ('qanda','HASHTAG','URL_LR'):
#                     continue
#                 if (dataset, train, test) == ('qanda','URL_LR','HASHTAG'):
#                     continue
#                 print('Area under the ROC Curve:', roc_auc_score(y_test[mask], preds, multi_class='ovo'))
#                 performance[dataset][train][test] = roc_auc_score(y_test[mask], preds, multi_class='ovo')
#             except Exception as e:
#                 print('Failed for:', train, test)
#                 raise(e)

In [12]:
list(all_combinations(fr_groundtruths))

[('URLa_FR', 'URLa_FR'),
 ('URLa_FR', 'URLb_FR'),
 ('URLb_FR', 'URLa_FR'),
 ('URLb_FR', 'URLb_FR')]

In [13]:
granularity = '_per_user'
feature = 'use'

for dataset in datasets:
    for train, test in itertools.chain(all_combinations(lr_groundtruths),all_combinations(fr_groundtruths)):
        print(dataset, train, test)
        train_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+train+'_'+granularity+'.pk')
        if not os.path.exists(train_path):
            continue
        test_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+test+'_'+granularity+'.pk')
        if not os.path.exists(test_path):
            continue

        y_train = load_vector(train_path).values
        mask_train = y_train >= 0

        y_test = load_vector(test_path).values
        mask_test = y_test >= 0
        mask = mask_train | mask_test
        
        # overlap_mask = mask_train & mask_test
        # print("Overlap F1:", f1_score(y_train[overlap_mask], y_test[overlap_mask],average='macro'))
        # overlap_f1[dataset][train][test] = f1_score(y_train[overlap_mask], y_test[overlap_mask],average='macro')
        # if train == 'POLITICIAN_LR' and test == 'POLITICIAN_1H_LR':
        #     mask = mask_train & mask_test
        if any(mask):
            # gt_f1[dataset][train][test] = f1_score(y_train[mask], y_test[mask],average='macro')

            # y = y_train[mask]
            y = y_train[mask_train]
            
            feature_path = os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk')
            X_orig = load_vector(feature_path)
            
            # X = X_orig[mask]
            X = X_orig[mask_train]
            et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
            if train == test:
                # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                # validation instance
                skf = StratifiedKFold(n_splits=5, shuffle=True)
                # getting the model validation predictions
                preds = cross_val_predict(et, X, y, cv=skf, method='predict_proba')
            else:
                # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                et.fit(X,y)
                X_text = X = X_orig[mask_test]
                preds = et.predict_proba(X_text)
            if train in fr_groundtruths:
                preds = preds[:,1]
            # evaluating the model
            print(dataset, train, test)
            print()
            try:
                if test == 'POLITICIAN_LR' and train == 'POLITICIAN_1H_LR':
                    N = preds.shape[0]
                    preds = np.c_[np.zeros(N),preds]
                print('Area under the ROC Curve:', roc_auc_score(y_test[mask_test], preds, multi_class='ovo'))
                performance[dataset][train][test] = roc_auc_score(y_test[mask_test], preds, multi_class='ovo')
            except Exception as e:
                print('Failed for:', train, test)
                raise(e)

qanda HASHTAG HASHTAG
qanda HASHTAG HASHTAG

Area under the ROC Curve: 0.8654219816217906
qanda HASHTAG URL_LR
qanda HASHTAG URL_LR

Area under the ROC Curve: 0.6945949670764197
qanda HASHTAG POLITICIAN_1H_LR
qanda HASHTAG POLITICIAN_1H_LR

Area under the ROC Curve: 0.5063032622265645
qanda HASHTAG PARTY_FOLLOWER_LR
qanda HASHTAG PARTY_FOLLOWER_LR

Area under the ROC Curve: 0.5255753138273361
qanda URL_LR HASHTAG
qanda URL_LR HASHTAG

Area under the ROC Curve: 0.7004600920193419
qanda URL_LR URL_LR
qanda URL_LR URL_LR

Area under the ROC Curve: 0.8182429692827992
qanda URL_LR POLITICIAN_1H_LR
qanda URL_LR POLITICIAN_1H_LR

Area under the ROC Curve: 0.541296884329371
qanda URL_LR PARTY_FOLLOWER_LR
qanda URL_LR PARTY_FOLLOWER_LR

Area under the ROC Curve: 0.5601043757611044
qanda POLITICIAN_1H_LR HASHTAG
qanda POLITICIAN_1H_LR HASHTAG

Area under the ROC Curve: 0.5710066742149217
qanda POLITICIAN_1H_LR URL_LR
qanda POLITICIAN_1H_LR URL_LR

Area under the ROC Curve: 0.5274442692141853
qan

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


qanda POLITICIAN_1H_LR POLITICIAN_1H_LR

Area under the ROC Curve: 0.8680318080660046
qanda POLITICIAN_1H_LR PARTY_FOLLOWER_LR
qanda POLITICIAN_1H_LR PARTY_FOLLOWER_LR

Area under the ROC Curve: 0.5538467793129266
qanda PARTY_FOLLOWER_LR HASHTAG
qanda PARTY_FOLLOWER_LR HASHTAG

Area under the ROC Curve: 0.6910726905163501
qanda PARTY_FOLLOWER_LR URL_LR
qanda PARTY_FOLLOWER_LR URL_LR

Area under the ROC Curve: 0.6221395288707942
qanda PARTY_FOLLOWER_LR POLITICIAN_1H_LR
qanda PARTY_FOLLOWER_LR POLITICIAN_1H_LR

Area under the ROC Curve: 0.7443955924659393
qanda PARTY_FOLLOWER_LR PARTY_FOLLOWER_LR
qanda PARTY_FOLLOWER_LR PARTY_FOLLOWER_LR

Area under the ROC Curve: 0.6181356494757789
qanda URLa_FR URLa_FR
qanda URLa_FR URLa_FR

Area under the ROC Curve: 0.6096006547598155
qanda URLa_FR URLb_FR
qanda URLa_FR URLb_FR

Area under the ROC Curve: 0.8313767230666785
qanda URLb_FR URLa_FR
qanda URLb_FR URLa_FR

Area under the ROC Curve: 0.9374195085268876
qanda URLb_FR URLb_FR
qanda URLb_FR URLb

In [21]:
col_order = ['HASHTAG','URL_LR','POLITICIAN_1H_LR','PARTY_FOLLOWER_LR','USER_FR','URLa_FR','URLb_FR']

pd.DataFrame(dict(performance[dataset])).reindex(index=col_order, columns=col_order)

# pd.DataFrame(dict(overlap_f1[dataset])).reindex(index=col_order, columns=col_order)

# pd.DataFrame(dict(gt_f1[dataset])).reindex(index=col_order, columns=col_order)

,HASHTAG,URL_LR,POLITICIAN_1H_LR,PARTY_FOLLOWER_LR,USER_FR,URLa_FR,URLb_FR
HASHTAG,0.861484,0.700460,0.571007,0.691073,NaN,NaN,NaN
URL_LR,0.694595,0.816256,0.527444,0.622140,NaN,NaN,NaN
POLITICIAN_1H_LR,0.506303,0.541297,0.872883,0.744396,NaN,NaN,NaN
PARTY_FOLLOWER_LR,0.525575,0.560104,0.553847,0.611889,NaN,NaN,NaN
USER_FR,NaN,NaN,NaN,NaN,0.655610,0.670019,0.770211
URLa_FR,NaN,NaN,NaN,NaN,0.854216,0.622464,0.937420
URLb_FR,NaN,NaN,NaN,NaN,0.864190,0.831377,0.701991


In [22]:
col_order = ['HASHTAG','URL_LR','POLITICIAN_1H_LR','PARTY_FOLLOWER_LR','URLa_FR','URLb_FR']
performance_dfs = {dataset: pd.DataFrame(dict(performance[dataset])).reindex(index=col_order, columns=col_order) for dataset in datasets}

In [23]:
performance_dfs['qanda']

,HASHTAG,URL_LR,POLITICIAN_1H_LR,PARTY_FOLLOWER_LR,URLa_FR,URLb_FR
HASHTAG,0.861484,0.700460,0.571007,0.691073,NaN,NaN
URL_LR,0.694595,0.816256,0.527444,0.622140,NaN,NaN
POLITICIAN_1H_LR,0.506303,0.541297,0.872883,0.744396,NaN,NaN
PARTY_FOLLOWER_LR,0.525575,0.560104,0.553847,0.611889,NaN,NaN
URLa_FR,NaN,NaN,NaN,NaN,0.622464,0.937420
URLb_FR,NaN,NaN,NaN,NaN,0.831377,0.701991


In [80]:
# gt_f1_dfs = {dataset: pd.DataFrame(gt_f1[dataset]).reindex(index=col_order, columns=col_order) for dataset in datasets}

In [81]:
# for dataset in datasets:
#     performance_dfs[dataset].to_csv('./'+dataset+'_cross_gt_auc.csv')
# for dataset in datasets:
#     gt_f1_dfs[dataset].to_csv('./'+dataset+'_cross_gt_f1.csv')

In [ ]:
performance_dfs['qanda'].to_csv(os.path.join(base_dir, 'data', '04_results', 'qanda_cross_gt_auc.csv'))